# **ML Coaching notebook**

## **Table of content**
1. [Installation](#Installation)
2. [Data exploration](#exploration)

## **Installation** <a name="Installation"></a>

If you have any problem to run BigQuery request, install **Google Cloud BigQuery** with this command

`%pip install --upgrade google-cloud-bigquery`

If you still have some errors, use the Big Query API with the --use_rest_api flag in your requests.

**Exemple:** `%%bigquery --use_rest_api SELECT * FROM *`

## **Data exploration** <a name="exploration"></a>

### First and Last event date

In [8]:
%%bigquery --use_rest_api
(SELECT FORMAT_DATE('%Y/%m/%d', PARSE_DATE('%Y%m%d', event_date)) AS event_date 
 FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` 
 ORDER BY event_date DESC 
 LIMIT 1)

UNION ALL

(SELECT FORMAT_DATE('%Y/%m/%d', PARSE_DATE('%Y%m%d', event_date)) AS event_date 
 FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` 
 ORDER BY event_date ASC 
 LIMIT 1)

Query is running:   0%|          |

Downloading:   0%|          |

,event_date
0,2020/11/01
1,2021/01/31


Latest event date is **2021/01/31** and earliest event date is **2020/11/01** ==> less than 1y data, not representative of seasonality

### List all event_name

In [7]:
%%bigquery --use_rest_api
SELECT event_name, COUNT(*) AS occurences 
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` 
GROUP BY event_name 
ORDER BY occurences DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,event_name,occurences
0,page_view,1350428
1,user_engagement,1058721
2,scroll,493072
3,view_item,386068
4,session_start,354970
5,first_visit,257462
6,view_promotion,190104
7,add_to_cart,58543
8,begin_checkout,38757
9,select_item,31007


### Count number of distinct user

In [10]:
%%bigquery --use_rest_api
SELECT COUNT(DISTINCT(user_pseudo_id)) as number_of_user_pseudo_id 
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` 

Query is running:   0%|          |

Downloading:   0%|          |

,number_of_user_pseudo_id
0,270154


### Get life time value of all users

In [17]:
%%bigquery --use_rest_api
SELECT
    DISTINCT(user_pseudo_id), 
    user_ltv.revenue, 
    user_ltv.currency 
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
ORDER BY user_ltv.revenue DESC LIMIT 100

Query is running:   0%|          |

Downloading:   0%|          |

,user_pseudo_id,revenue,currency
0,2664459.3880988628,3360.0,USD
1,81036435.1157569602,1530.0,USD
2,15046526.4562004600,1424.0,USD
3,8083450.3844124226,1409.0,USD
4,8083450.3844124226,1281.0,USD
...,...,...,...
95,3883375.8924071243,517.0,USD
96,63264809.7446857295,516.0,USD
97,2808102.2330574481,516.0,USD
98,8083450.3844124226,514.0,USD


### How to unnest fields ?

To unnest a field, add `UNNEST(field_name) as variable_name`

**Exemple:**

In [22]:
%%bigquery --use_rest_api
SELECT 
    DISTINCT(i.item_name) 
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST(items) as i
LIMIT 100

Query is running:   0%|          |

Downloading:   0%|          |

,item_name
0,Noogler Android Figure
1,Android Large Trace Journal Black
2,Google Cork Base Tumbler
3,YouTube Standards Zip Hoodie Black
4,Google Youth F/C Pullover Hoodie
...,...
95,Supernatural Paper Tote
96,Google Crew Striped Athletic Sock
97,Google Mountain View Campus Zip Hoodie
98,Google Sunnyvale Campus Unisex Tee


### is page location containing only links ? 

In [29]:
%%bigquery --use_rest_api
SELECT COUNT(*) FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST(event_params) as ep
WHERE ep.key = "page_location" AND ep.value.string_value NOT LIKE "%http%"

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,0


It's look like the field contain only links 

### The session_engaged field is not usable

In [30]:
%%bigquery --use_rest_api
SELECT DISTINCT(ep.value.string_value) FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST(event_params) as ep
WHERE ep.key = "session_engaged"

Query is running:   0%|          |

Downloading:   0%|          |

,string_value
0,0
1,None
2,1


### Engagement time of users

In [34]:
%%bigquery --use_rest_api
SELECT DISTINCT(ep.value.int_value) FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST(event_params) as ep
WHERE ep.key = "engagement_time_msec" LIMIT 1000

Query is running:   0%|          |

Downloading:   0%|          |

,int_value
0,50127
1,1228
2,1036
3,419
4,578
...,...
995,6257
996,26439
997,7741
998,8279


### Country of users

In [38]:
%%bigquery --use_rest_api
SELECT geo.country, COUNT(*) as num_orders FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE event_name = "add_to_cart"
GROUP BY geo.country
ORDER BY num_orders DESC

Query is running:   0%|          |

Downloading:   0%|          |

,country,num_orders
0,United States,26180
1,India,5263
2,Canada,4617
3,United Kingdom,1764
4,Spain,1251
...,...,...
104,Latvia,3
105,Lebanon,2
106,Kosovo,2
107,Honduras,2


### List page location

In [46]:
%%bigquery --use_rest_api
SELECT ep.value.string_value, COUNT(*) AS num FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST(event_params) as ep
WHERE event_name = "add_to_cart" AND ep.key = "page_location"
GROUP BY ep.value.string_value
ORDER BY num DESC

Query is running:   0%|          |

Downloading:   0%|          |

,string_value,num
0,https://shop.googlemerchandisestore.com/Google...,9167
1,https://shop.googlemerchandisestore.com/Google...,7690
2,https://shop.googlemerchandisestore.com/store....,6532
3,https://shop.googlemerchandisestore.com/Google...,3802
4,https://shop.googlemerchandisestore.com/Google...,3076
5,https://shop.googlemerchandisestore.com/Google...,2931
6,https://shop.googlemerchandisestore.com/Google...,2863
7,https://shop.googlemerchandisestore.com/Google...,1972
8,https://shop.googlemerchandisestore.com/Google...,1914
9,https://shop.googlemerchandisestore.com/asearc...,1825
